In [1]:
import torch 
import torch.nn as nn
import lightning as L

import xarray as xr
import numpy as np

import matplotlib.pyplot as plt

from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger
from BalticRiverPrediction.convLSTM import ConvLSTM

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split

from BalticRiverPrediction.BaltNet import BaltNet, LightningModel
from BalticRiverPrediction.BaltNet import AtmosphereDataModule, AtmosphericDataset
from BalticRiverPrediction.sharedUtilities import read_netcdfs, preprocess

In [2]:
# Set seed for reproducible
L.seed_everything(123)

# Use available tensor cores
torch.set_float32_matmul_precision("high")

# X 
datapath="/silor/boergel/paper/runoff_prediction/data"
datapathPP="/fast/boergel/paper/runoff_prediction"

# runoff = xr.open_dataset(f"{datapathPP}/runoff.nc").load()
# runoff = runoff.roflux

DataRain = xr.open_dataset(f"{datapathPP}/rain2.nc")
DataRain = DataRain.sel(time=slice("2011", "2014"))
DataRain = DataRain.RAIN.squeeze()

DataShumi = xr.open_dataset(f"{datapathPP}/shumi.nc")
DataShumi = DataShumi.sel(time=slice("2011", "2014"))
DataShumi = DataShumi.shumi.squeeze()

DataWindMagnitude = xr.open_dataset(f"{datapathPP}/windxy.nc")
DataWindMagnitude['wind_magnitude'] = (DataWindMagnitude['windx']**2 + DataWindMagnitude['windy']**2)**0.5
DataWindMagnitude = DataWindMagnitude.sel(time=slice("2011", "2014"))
DataWindMagnitude = DataWindMagnitude.wind_magnitude.squeeze(dim="height", drop=True)

assert DataShumi.time[0] == DataRain.time[0] == DataWindMagnitude.time[0]
assert len(DataShumi.time) == len(DataRain.time) == len(DataWindMagnitude.time)

data = xr.merge([DataRain, DataShumi, DataWindMagnitude])

runoff = read_netcdfs(
    f"{datapath}/runoffData/combined_fastriver_*.nc",
    dim="river",
    transform_func= lambda ds:ds.sel(time=slice(str(2011), str(2014))).roflux.resample(time="1D").mean(),
    cftime=False
    )  

assert len(runoff.time) == len(data.time)

modelParameters = {
    "input_dim": 3,
    "hidden_dim": 6,
    "kernel_size": (7,7),
    "num_layers": 2,
    "batch_first": True,
    "bias": True,
    "return_all_layers": False,
    "dimensions": (191, 206),
    "input_size": 30
}

Global seed set to 123
100%|█████████████████████████████████████████████████████████████████████████| 97/97 [00:59<00:00,  1.62it/s]


Config defines the setup of the model and the input data it expects. This will also affect the dataLoader. DataLoader automatically create batches of data and also put the data onto the GPU efficiently.

In [3]:
# Loads the atmospheric data in batches
dataLoader = AtmosphereDataModule(
atmosphericData=data,
runoff=runoff,
batch_size=64,
input_size=modelParameters["input_size"],
num_workers=8
)

num_epochs = 40
pyTorchBaltNet = BaltNet(modelPar=modelParameters)


In [4]:
LightningBaltNet = LightningModel.load_from_checkpoint(
        checkpoint_path=f"/silor/boergel/paper/runoff_prediction/data/modelWeights/ModelWeight_H6_K(7x7)_I30_L2TopOne-v1.ckpt",
        learning_rate=1e-6,
        model=pyTorchBaltNet
    )

In [5]:
dataDataset = AtmosphericDataset(
    atmosphericData=data.sel(time=slice(str(2011), str(2014))),
    runoff=runoff,
    input_size=modelParameters["input_size"],
)

In [6]:
predict_dataloader = DataLoader(
    dataDataset,
    batch_size=64,
    shuffle=False,
    drop_last=True
    )

In [7]:
trainer = L.Trainer(
    inference_mode=True,
    accelerator="cuda",
    devices=1,
    deterministic=True,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
predictions = trainer.predict(LightningBaltNet, dataloaders=predict_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/silos/conda_packages/boergel/miniconda3_4.12.0/OS_15.4/conda_env/BaltNet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 58.00 MiB (GPU 0; 44.42 GiB total capacity; 3.08 GiB already allocated; 13.75 MiB free; 3.09 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
#dataDataset.runoffDataStats

In [ ]:
concatenated_predictions = torch.cat(predictions, dim=0)
rescaledRunoffData = concatenated_predictions*dataDataset.runoffDataStats[1].data+dataDataset.runoffDataStats[0].data

In [ ]:
origianlRunoffData = runoff.transpose("time", "river")
origianlRunoffData = origianlRunoffData.resample(time="1D").mean()

In [ ]:
rescaledRunoffDataDataset = xr.DataArray(
    data = rescaledRunoffData.data,
    coords={
        "time": origianlRunoffData.time[modelParameters["input_size"]:],
        "river": origianlRunoffData.river
    }
)


In [ ]:
f, ax = plt.subplots(1, figsize=(12,12))
rescaledRunoffDataDataset.isel(river=1).plot(ax=ax, label = "Predict", alpha=.5)
origianlRunoffData.isel(river=1).plot(ax=ax, label="Original", alpha=0.5)
plt.legend()

In [ ]:
f, axs = plt.subplots(4,4, figsize=(15, 15))

for i, ax in enumerate(axs.ravel()):
    ax.set_title(f"River {i+1}")
    rescaledRunoffDataDataset.isel(river=i).plot(ax=ax, label = "Predict", alpha=.5)
    origianlRunoffData.isel(river=i).plot(ax=ax, label="Original", alpha=0.5)
    ax.legend()
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    
plt.tight_layout()


In [ ]:
class TaylorDiagram(object):
    """
    Taylor diagram.
    Plot model standard deviation and correlation to reference (data)
    sample in a single-quadrant polar plot, with r=stddev and
    theta=arccos(correlation).
    """
    def __init__(self, refdata, ax,  *args, **kwargs):
        # memorize reference data
        self.refdata = refdata
        self.ax = ax
        refstd = refdata.std(ddof=1) 
        rlim = [0.0, 1.2*refstd]
        thetalim = [0.0, 0.5*np.pi]
        if ax != None:
            # Add reference point and stddev contour
            try:
                kwargs["marker"]
            except:
                kwargs["marker"] = 'o'
            try:
                kwargs["color"]
            except:
                kwargs["color"] = "black"
            try:
                kwargs["ms"]
            except:
                kwargs["ms"] = 10     
            try:
                kwargs["label"]
            except:
                kwargs["label"] = "reference"                     
            ax.plot([0], refstd, *args, ls='', zorder=0, **kwargs)
            ax.grid(True, linestyle="--")
            
            ax.set_xlabel("St. dev.")
            ax.xaxis.set_label_coords(0.5, -0.1)
            ax.set_xlim(thetalim)
            ax.set_ylim(rlim)
            self.rlim = rlim
            self.thetalim = thetalim
        self.model_std = {"reference" : refstd}
        self.corrcoef = {"reference" : 1.0}
        self.rms = {"reference" : 0.0}
    def add_sample(self, model_data, *args, **kwargs):
        """
        Add sample to the Taylor
        diagram. *args* and *kwargs* are directly propagated to the
        `Figure.plot` command.
        """
        model_std = model_data.std(ddof=1)
        corrcoef = np.corrcoef(model_data, self.refdata)[0, 1]
        rms = np.sqrt(self.model_std["reference"]**2 + model_std**2 - 2.0*self.model_std["reference"]*model_std*np.cos(np.arccos(corrcoef)))
        try:
            label = kwargs["label"]
        except:
            label = "model"+str(len(self.model_std.keys())) 
        if self.ax != None:
            self.ax.plot(np.arccos(corrcoef), model_std,
                            *args, **kwargs)  # (theta, radius)
            if model_std > self.rlim[1]:
                self.rlim = [0, 1.2 * model_std]
                self.ax.set_ylim(*self.rlim)
            if np.arccos(corrcoef) > self.thetalim[1]:
                self.thetalim[1] = 1.2 * np.arccos(corrcoef)
                self.ax.set_xlim(*self.thetalim)                       
        self.model_std[label] = model_std
        self.corrcoef[label] = corrcoef
        self.rms[label] = rms
        #self.ax.set_ylim(rlim) 
    def get_samples(self):
        return self.model_std, self.corrcoef, self.rms
    def finalize(self):
        
        if self.ax is None:
            return 
        self.ax.text(0.5*(self.thetalim[1]-self.thetalim[0]), (1.0+0.03*(self.thetalim[1]-self.thetalim[0])**2)*self.rlim[1],"Correlation", rotation=0.5*(self.thetalim[1]-self.thetalim[0])*180.0/np.pi-90.0)
        rs, ts = np.meshgrid(np.linspace(*self.rlim), np.linspace(*self.thetalim))
        rms = np.sqrt(self.model_std["reference"]**2 + rs**2 - 2.0*self.model_std["reference"]*rs*np.cos(ts))
        self.ax.contourf(ts, rs, rms, 9, alpha=0.3, cmap="RdYlGn_r")
        contours = self.ax.contour(ts, rs, rms, 9, linestyles="--", linewidths=1, alpha=0.5, colors="grey")
        if self.model_std["reference"] > 100.0 or self.model_std["reference"] < 0.1:
            fmt = '%3.2e'
        else:
            fmt = '%3.2f'
        self.ax.clabel(contours, inline=False, fmt=fmt, colors="black")
        t = np.linspace(*self.thetalim)
        r = np.zeros_like(t) + self.model_std["reference"]
        self.ax.plot(t, r, 'k--', label='_')
        xticks = [1.0, 0.99, 0.95, 0.9, 0.8, 0.7, 0.6, 0.4, 0.2, 0.0]
        if self.thetalim[1] > 0.5*np.pi:
            for x in [-0.2, -0.4, -0.6, -0.8, -0.9, -0.95, -0.99 -1.0]:
                if np.arccos(x) > self.thetalim[1]:
                    break
                xticks += [x]
        self.ax.set_xticks(np.arccos(xticks))
        self.ax.set_xticklabels(xticks)

In [ ]:
orignalTaylor = origianlRunoffData.sel(time=slice(rescaledRunoffDataDataset.time.min(), None)).isel(river=0)
modelPrediction = rescaledRunoffDataDataset.isel(river=0)

In [ ]:
f, ax = plt.subplots(1)
ax.plot(orignalTaylor.data)
ax.plot(modelPrediction.data)

In [ ]:
f, axes = plt.subplots(6,6, figsize=(16,16), subplot_kw={'projection': 'polar'})

for i,ax in enumerate(axes.ravel()):
    orignalTaylor = origianlRunoffData.sel(time=slice(rescaledRunoffDataDataset.time.min(), None)).isel(river=i)
    modelPrediction = rescaledRunoffDataDataset.isel(river=i)   
    diagram = TaylorDiagram(orignalTaylor.data, ax)
    diagram.add_sample(modelPrediction.data, marker='*', ms=10, ls='', color="black", label="River 0")
    diagram.finalize()